In [1]:
import time
import csv
from datetime import datetime
import pytz

def start_task():
    return time.time()

def end_task(start_time):
    end_time = time.time()
    elapsed_time = end_time - start_time
    return elapsed_time

def log_task(task_name, elapsed_time):
    #brt = pytz.timezone('America/Sao_Paulo')
    
    pt = pytz.timezone('America/Los_Angeles')
    today_brt = datetime.now(pt).date()
    with open('task_log.csv', 'a', newline='') as csvfile:
        fieldnames = ['date', 'task_name', 'elapsed_time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writerow({'date': today_brt, 'task_name': task_name, 'elapsed_time': elapsed_time})


In [2]:
#Função monitoramento de inicio e fim de task
import time
import csv
import pyautogui
from datetime import date

import time

def monitor_buttons(start_button, acquire_button, nrt_image):
    print("Monitorando os botões 'Começar' e 'Submeter'...")

    while True:
        try:
            start_button_location = pyautogui.locateOnScreen(start_button, confidence=0.85)
            if start_button_location:
                start_time = start_task()
                print("Botão 'Começar' detectado. Iniciando tarefa...")

                while True:
                    try:
                        if monitor_overtime():
                            submit_task()
                            
                        acquire_button_location = pyautogui.locateOnScreen(acquire_button, confidence=0.9)
                        nrt_message = pyautogui.locateOnScreen(nrt_image, confidence=0.9)
                        if acquire_button_location or nrt_message:
                            elapsed_time = end_task(start_time)
                            print("Botão 'Submeter' detectado. Finalizando tarefa...")
                            return elapsed_time
                    except OSError:
                        print("Screen grab failed in the inner loop, trying again...")
                        time.sleep(1)  # Wait a bit before trying again

        except OSError:
            print("Screen grab failed in the outer loop, trying again...")
            time.sleep(1)  # Wait a bit before trying again



In [3]:
#Função que submete a task
import pyautogui
import time

def submit_task():


    quantidade_rolagem = -10000  # Quantidade de rolagem desejada
    
    time.sleep(1)
    
    # Encontrar a posição do botão
    botao_pos = pyautogui.locateOnScreen('submit.png', confidence=0.80)

    if botao_pos is None:
        botao_pos = pyautogui.locateOnScreen('submitv2.png', confidence=0.80)


    if botao_pos is not None:
        
        pyautogui.scroll(quantidade_rolagem)
        time.sleep(1)
        # Obter o centro do botão
        botao_centro = pyautogui.center(botao_pos)
        
        # Mover o mouse para o centro do botão
        pyautogui.moveTo(botao_centro)
        
        # Clicar no botão
        pyautogui.click()
        
        # Aguardar um momento para a ação ser concluída
        time.sleep(0.5)
    else:
        print("Botão não encontrado na tela.")


In [4]:
'''
#Função de monitoramento de overtime para submite de task
import time
import csv
import pyautogui
from datetime import date

import time

def monitor_overtime():
    over_timer_image = 'overtime.png'
    
    over_time_location = pyautogui.locateOnScreen(over_timer_image, confidence=0.95)
    if over_time_location:
        print('Achou overtime')
        return True
    else:
        return False



'''


"\n#Função de monitoramento de overtime para submite de task\nimport time\nimport csv\nimport pyautogui\nfrom datetime import date\n\nimport time\n\ndef monitor_overtime():\n    over_timer_image = 'overtime.png'\n    \n    over_time_location = pyautogui.locateOnScreen(over_timer_image, confidence=0.95)\n    if over_time_location:\n        print('Achou overtime')\n        return True\n    else:\n        return False\n\n\n\n"

In [5]:
import cv2
import numpy as np
import pyscreenshot as ImageGrab

def monitor_overtime():
    # Caminho da imagem de template (overtime)
    over_timer_image_path = 'overtime.png'
    template = cv2.imread(over_timer_image_path, cv2.IMREAD_GRAYSCALE)
    
    # Capturar a tela inteira
    screenshot = ImageGrab.grab()
    screenshot_np = np.array(screenshot) 
    main_image = cv2.cvtColor(screenshot_np, cv2.COLOR_RGB2GRAY)
    
    # Usar o método de template matching
    result = cv2.matchTemplate(main_image, template, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(result)
    
    # Definir um limiar de confiança
    threshold = 0.98
    if max_val > threshold:
        print('Achou overtime')
        return True
    else:
        return False


# Testar a função
if monitor_overtime():
    print("Overtime detectado!")
else:
    print("Overtime não detectado.")




Overtime não detectado.


In [7]:
#Roda a função continuamente o dia todo
import time
start_button = 'timer.png'
acquire_button = 'acquire_button.png'
nrt_image = 'nrt.png'

task_counter = 1

try:
    while True:
        time.sleep(0.5)
        task_name = f"Tarefa {task_counter}"
        elapsed_time = monitor_buttons(start_button, acquire_button, nrt_image)
        print(f"Tempo gasto na {task_name}: {elapsed_time} segundos")
        log_task(task_name, elapsed_time)

        # Atualiza o contador e repete o processo para a próxima tarefa
        task_counter += 1
except KeyboardInterrupt:
    print("Programa interrompido pelo usuário.")


Monitorando os botões 'Começar' e 'Submeter'...
Botão 'Começar' detectado. Iniciando tarefa...
Achou overtime
Achou overtime
Botão não encontrado na tela.
Achou overtime
Botão não encontrado na tela.


In [ ]:

import csv
from collections import defaultdict
from datetime import datetime, timedelta, date

def seconds_to_hours_minutes(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    return hours, minutes

def sum_daily_tasks(file_name):
    daily_tasks = defaultdict(float)

    with open(file_name, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        for row in reader:
            day = row['date']
            elapsed_time = float(row['elapsed_time'])
            daily_tasks[day] += elapsed_time

    return dict(daily_tasks)


from datetime import date, timedelta

def week_start_end(task_date):
    start = task_date - timedelta(days=(task_date.weekday() + 1) % 7)  # domingo
    end = start + timedelta(days=6)  # sábado
    return start, end

def sum_weekly_tasks(file_name, current_week_start, current_week_end):
    weekly_tasks = defaultdict(float)
    
    with open(file_name, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        for row in reader:
            task_date = datetime.strptime(row['date'], "%Y-%m-%d").date()
            if current_week_start <= task_date <= current_week_end:
                start, end = week_start_end(task_date)
                week_range = f"{start} - {end}"
                elapsed_time = float(row['elapsed_time'])
                weekly_tasks[week_range] += elapsed_time

    return dict(weekly_tasks)

# Obtenha a data de início e término da semana corrente
today = date.today()
current_week_start, current_week_end = week_start_end(today)



# Exemplo de uso:

task_log_file = 'task_log.csv'
daily_tasks = sum_daily_tasks(task_log_file)

print("Tempo gasto diário nas tarefas:")
for day, elapsed_time in daily_tasks.items():
    hours, minutes = seconds_to_hours_minutes(elapsed_time)
    print(f"{day}: {hours} h {minutes} m")

task_log_file = 'task_log.csv'
weekly_tasks = sum_weekly_tasks(task_log_file, current_week_start, current_week_end)


print("Tempo gasto semanal ( semana corrente ) nas tarefas:")
for week, elapsed_time in weekly_tasks.items():
    hours, minutes = seconds_to_hours_minutes(elapsed_time)
    print(f"{week}: {hours} h {minutes} m")



Tempo gasto diário nas tarefas:
2023-05-05: 0 h 3 m
2023-05-06: 1 h 5 m
2023-05-07: 0 h 43 m
2023-05-08: 3 h 19 m
2023-05-10: 1 h 0 m
2023-05-11: 0 h 46 m
2023-05-12: 1 h 46 m
2023-05-13: 0 h 26 m
2023-05-14: 0 h 49 m
2023-05-15: 1 h 19 m
2023-05-16: 0 h 40 m
2023-05-17: 0 h 17 m
2023-05-18: 1 h 11 m
2023-05-19: 1 h 27 m
2023-05-20: 1 h 7 m
2023-05-22: 0 h 23 m
2023-05-23: 0 h 54 m
2023-05-24: 0 h 44 m
2023-05-26: 2 h 16 m
2023-05-29: 0 h 29 m
2023-05-30: 1 h 16 m
2023-05-31: 3 h 22 m
2023-06-01: 1 h 53 m
2023-06-02: 0 h 16 m
2023-06-03: 0 h 14 m
2023-07-03: 0 h 32 m
2023-06-04: 1 h 21 m
2023-06-05: 2 h 17 m
2023-06-06: 6 h 26 m
2023-06-11: 0 h 44 m
2023-06-13: 2 h 28 m
2023-06-14: 3 h 51 m
2023-06-16: 1 h 51 m
2023-06-17: 0 h 15 m
2023-06-18: 1 h 28 m
2023-06-19: 0 h 38 m
2023-06-20: 0 h 31 m
2023-06-21: 2 h 26 m
2023-06-22: 2 h 45 m
2023-06-23: 1 h 50 m
2023-06-24: 0 h 15 m
2023-06-25: 0 h 42 m
2023-06-26: 1 h 43 m
2023-06-27: 0 h 39 m
2023-06-28: 2 h 30 m
2023-06-29: 0 h 10 m
2023-0